<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Recherche d'Information et traitement de données massives </center>
    <center> Lab 2 : Modèles de Recherche </center>
<hr style=" border:none; height:3px;">
</h1>




# Partie 1


L' objectif de cette partie est de mettre en oeuvre les différents modèles de recherche vus en cours 2 (Booléen et Vectoriel) sur le corpus TIME.

## Préambule

**Pour ceux qui travaillent sur Colab (depuis leur répertoire local), suivez ces intructions:**


1. Dans le dossier Lab2_ModelesDeRecherche téléchargé depuis git, vous trouverez une archive Archive_Lab2.zip 
2.   Dans Colab, ouvrir le panneau Fichiers (c'est-à-dire cliquez sur le logo en forme de dossier à gauche) et cliquez sur le premier bouton en haut à gauche vous permettant de télécharger. Téléchargez ainsi l'Archive_Lab2.zip dans votre colab. Si besoin actualiser jusqu'à voir l'archive apparaître dans le panneau. 
3.   Exécutez les deux cellules de code suivantes pour dézipper l'archive et installer la librairie nltk (nous en aurons besoin dans la section sur le modèle vectoriel). 

**Pour les autres travaillant sur leur machine vous avez tout ce qu'il faut dans le dossier Lab2_ModelesDeRecherche et donc rien à faire de plus**. 

In [ ]:
!unzip Archive_Lab2

In [ ]:
!pip install nltk
import nltk
nltk.download()

### Chargement de l'index

1. Charger l'index inversé de la collection TIME construit et sauvegardé sous la forme d'un fichier dans le LAB1. 

Les différents index sont disponibles [ici](./Index) (dossier Index). Nous avons aussi mis dans le répertoire [Utils](./Utils) un ensemble de modules python permettant de charger les index.

Note: les personnes travaillant sur Colab, les dossiers sont dans Archive_Lab2

In [ ]:
from Utils.saveandload_pickle import *
from Archive_Lab_2.Utils.saveandload_pickle import *  #(pour les personnes sur Colab)

# A completer

###  Le Modèle Booléen

**Représentation des requêtes**

Une des premières étapes pour la mise en oeuvre du modèle booléen est la représentation d'une requête. En effet, dans le modèle booléen, les requêtes sont des expressions booléennes pouvant être définies à l'aide des opérateurs logiques `OR`, `NOT` et `AND` comme par exemple la requête ` U.S AND War OR France`.  Nous avons vu dans le cours qu'il pouvait être intéressant de représenter les requêtes par leur [forme normale conjonctive](https://fr.wikipedia.org/wiki/Forme_normale_conjonctive) (conjonction de disjonctions) afin d'optimiser leur traitement à l'aide de l'index inversé. Dans la suite, nous supposerons donc que les requêtes sont exprimées sous cette forme. 

2- **Quelle structure de données informatique proposez-vous pour representer une requête sous forme normale conjonctive et faciliter son traitement ?** Puis, à partir de cette représentation, quelle stratégie mettre en place pour évaluer l'expression booléenne associée ?

Indice :


<center><img src='https://drive.google.com/uc?export=view&id=1pG65-EId7dT_FHM-Gh4GPimjNaCgg_xZ' width=400></center>

Une manière de faire pourrait être de récupérer les littéraux `U.S`et `France `et de leur appliquer l'opérateur `OR` puis de récupérer le littéral `War`de lui appliquer `NOT` et enfin d'appliquer l'opérateur `AND` aux deux résultats des évaluations précédentes. Si on écrit la formule selon cette stratégie, on obtient :
` U.S France OR War NOT AND`. 
Cela correspond à la notation post-fixe de la formule. Cette notation, aussi connue sous le nom de [notation polonaise inversée](https://fr.wikipedia.org/wiki/Notation_polonaise_inverse) (en hommage à son créateur Jan Łukasiewicz grand Logicien et Philosophe Polonais) est très pratique pour l'évaluation de la formule. Elle permet aussi de ne pas utiliser de parenthèses sans ambiguité.

Transformer une expression en sa représentation sous forme d'arbre ou en sa notation post-fixe est un beau problème d'algorithmique et de programmation et vous pourrez y réflechir à la maison. Dans le Lab, pour nous éviter de passer trop de temps sur cette question, nous utiliserons la bibliothèque [`tt`](http://tt.brianwel.ch/en/latest/index.html) qui permet de travailler avec des expressions en logique booléenne sous python.

Installer la bibliothéque en executant la commande ci-dessous.


In [ ]:
!pip install ttable

Le code ci-dessous vous permet de transformer une requête exprimée sous la forme d'une chaîne de caractères en une formule booléeen.

In [ ]:
from tt import BooleanExpression

b = BooleanExpression('(War or France) and (not C)')
print(b)   

L'instruction ci-dessous permet de récupérer l'arbre représentant la formule

In [ ]:
b.tree
print(b.tree)

Il est aussi possible de récupérer l'ensemble des tokens avec l'instruction ci-dessous.

In [ ]:
 b.tokens

De même, il est possible de récupérer l'ensemble des tokens dans l'ordre post-fixe.

In [ ]:
b.postfix_tokens

Dans la suite, on considérera donc deux cas:

+ **1er cas** : les requêtes sont données en langage naturel (comme par exemple dans le fichier [TIME.QUE](./Data/Time/TIME.QUE)), fournies avec la collection TIME et il faudra donc transformer cette requête en expression logique. En particulier, on considèrera que l'espace correspond à l'opérateur `AND`. Par exemple, la requête `KENNEDY ADMINISTRATION PRESSURE` correspond à la requête `kennedy AND administration AND pressure`. 

3- Ecrire une fonction `def transformation_query_to_boolean(query)` qui permet de transformer une requête en langage naturel sous sa forme logique.


In [ ]:
def transformation_query_to_boolean(query):
    
    # A completer
    return boolean_query


In [ ]:
#Test


transformation_query_to_boolean("KENNEDY ADMINISTRATION PRESSURE")

 + **2ème cas** : les requêtes sont fournies comme une expression logique (par exemple `'(War or France) and (not C)'`) et il s'agit de la transformer sous une forme post-fixe pour son evaluation.

4- En utilisant la bibliothèque `tt` comme montré précedemment, écrire une fonction permettant de transformer une requête en son ensemble de tokens ordonnés par ordre post-fixe.


In [ ]:
from tt import BooleanExpression

def transformation_query_to_postfixe(query):
    #a compléter

In [ ]:
#Test

transformation_query_to_postfixe('(War or France) and (not C)')

**Traitement des requêtes**

Il s'agit ici d'écrire une fonction `processing_boolean_query_with_inverted_index(booleanOperator,query, inverted_index)` qui permet de renvoyer la liste de documents pertinents pour la requête `query`. `booleanOperator`est l'ensemble des opérateurs booléens considérés et `inverted_index` est l'index inversé de la collection considérée.

Quelques indications pour vous aider :
 + L'utilisation d'une structure de données [Pile](https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks) et de la notation polonaise inversée peut être utile pour l'évaluation de la requête complète.
 + Revoir le cours 2 sur le modèle booléen pour l'utilisation de l'index inversé dans ce cas.
 
**On suppose que les listes de postings de l'index inversé sont triées par doc_id croissant**

5- Pour faciliter l'écriture de cette fonction, nous allons d'abord écrire les fonctions :
+ `def merge_and_postings_list(posting_term1,posting_term2)` qui applique l'opérateur AND à deux postings listes : intersection de posting liste (c.f. cours) ;
+ `def merge_or_postings_list(posting_term1,posting_term2)` qui applique l'opérateur OR à deux postings listes : union de posting liste (c.f. cours) ;
+ `def merge_and_not_postings_list(posting_term1,posting_term2)`qui calcule posting_term1 AND NOT posting_term2.

In [ ]:
def merge_and_postings_list(posting_term1,posting_term2):
    result=[]
    # A completer
    return result

# Test 
print("Postings du terme {} : {}".format('NASSAU',document_index['NASSAU']))
print("Postings du terme {} : {}".format('FRANCE',document_index['FRANCE']))
print("Test du AND")
res = merge_and_postings_list(document_index["NASSAU"], document_index["FRANCE"])
print(res)

In [ ]:
def merge_or_postings_list(posting_term1,posting_term2):
    result=[]
    # A completer
    return result


# Test 
print("Postings du terme {} : {}".format('NASSAU',document_index['NASSAU']))
print("Postings du terme {} : {}".format('FRANCE',document_index['FRANCE']))
print("Test du OR")
res = merge_or_postings_list(document_index["NASSAU"], document_index["FRANCE"])
print(res)

In [ ]:
def merge_and_not_postings_list(posting_term1,posting_term2):
    result=[]
    # A completer
    return result


# Test
print("Postings du terme {} : {}".format('NASSAU',document_index['NASSAU']))
print("Postings du terme {} : {}".format('FRANCE',document_index['FRANCE']))
print("Test du AND_NOT")
res = merge_and_not_postings_list(document_index["NASSAU"], document_index["FRANCE"])
print(res)


6- En utilisant les fonctions implémentées dans la question précédente, écrire la fonction `boolean_operator_processing_with_inverted_index(BoolOperator,posting_term1,posting_term2)` qui applique l'opérateur `BoolOperator` parmi (AND, OR et AND NOT) à deux posting listes.

In [ ]:
def boolean_operator_processing_with_inverted_index(BoolOperator,posting_term1,posting_term2):
    result=[]
    # A completer
    return result

In [ ]:
#Test

BooleanOperator = {"AND", "OR", "NOT"}
print("Postings du terme {} : {}".format('NASSAU',document_index['NASSAU']))
print("Postings du terme {} : {}".format('FRANCE',document_index['FRANCE']))


print("Test de la fonction globale")
#test de la fonction précédente
result = boolean_operator_processing_with_inverted_index('OR',document_index['NASSAU'],document_index['FRANCE'])
print(result)

7- Ecrire la fonction `def processing_boolean_query_with_inverted_index(booleanOperator,query, inverted_index)` qui permet de renvoyer la liste de documents pertinents pour la requête `query`. `booleanOperator`est l'ensemble des opérateurs booléens considérés et `inverted_index` est l'index inversé de la collection considérée.


In [ ]:
def processing_boolean_query_with_inverted_index(booleanOperator,query, inverted_index):
    # A completer

    

In [ ]:
#Test

#A compléter

## Modèle vectoriel

**Représentation des requêtes et des documents**




Dans le modèle vectoriel, les requêtes et les documents ont le même type de représentation, un vecteur pondéré dans l'espace des termes d'indexation. Il est donc nécessaire de faire subir à votre requête, souvent exprimée en langage naturel, les mêmes traitements que ceux appliqués aux documents lors de la phase d'indexation  soit :
+ L'étape de segmentation ;
+ L'étape de filtrage ;
+ L'étape de normalisation.

Cela permettra notamment de garantir que l'espace de représentation de la requête et du document sont les mêmes.

8- Ecrire une fonction `pre_processed_query(query)`qui prend en entrée une requête sous la forme d'une chaîne de caractères et qui renvoie la requête sous la forme d'une liste de termes filtrés et normalisés. On pourra pour cela ré-utiliser les fonctions écrites dans le Lab 1 et fournies dans le fichier [Lab1.py](./Utils/Lab1.py) du répertoire [Utils](./Utils).



In [ ]:
from Utils.Lab1 import * 
#from Archive_Lab2.Utils.Lab1 import * (pour Colab)

def pre_processed_query(query):
    # A completer
   

 

In [ ]:
# Test
pre_processed_query("KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP")

Un choix doit cependant être fait sur la manière de pondérer les différents termes de la requête et des documents lors de la construction de leurs représentations. En particulier, différents schémas de pondération sont possibles.
Il peut être intéressant de pouvoir comparer ces différents types de pondération ainsi, dans la suite nous nous intéresserons aux schémas de pondérations suivants :
+ **Pondération binaire** comme dans le modèle booléen ;
+ **Pondération `tf`** avec  $w_{id} = tf_{t_i,d}$ où $d$ peut être un document ou une requête ;
+ **Pondération `tf-idf`** avec donc $w_{id} = tf_{t_i,d} \times idf_{t_i}$ où $idf_{t_i} = \log{\frac{N}{df_{t_i}}}$ et $N$ étant le nombre de documents dans la collection ;
+ **Pondération fréquentielle normalisée** qui consiste à normaliser les fréquences des termes apparaissant dans un document $d$ par la fréquence maximale dans le document $tf_{max_d} = \max_{t \in d} tf_{t,d}$ soit pour le terme $p_{tf_{t_i,d}}$ correspondant à la fréquence dans la collection la formule :
$ p_{tf_{t_i,d}} = 0.5 + 0.5 \times \frac{tf_{t_i,d}}{tf_{max_d}}$ ;
+ **Pondération logarithmique des termes** avec $p_{tf_{t_i,d}} = 1 + \log{tf_{t_i,d}}$ si $tf_{t_i,d} > 0$ et $0$ sinon ;
+ **Pondération logarithmique des termes normalisée** qui consiste à normaliser la caractéristique logarithmique précédente par une quantité dépendant de la moyenne $moy_{tf_d} = \sum_{i=1}^{V}\frac{tf_{t_i,d}}{|d|}$ ($|d|$ est le nombre de termes uniques du document $d$) des caractéristiques fréquentielles d'un document $d$ soit $ p_{tf_{t_i,d}} = \frac{1 + \log{tf_{t_i,d}}}{1 + \log{moy_{tf_d}}}$ ;
+ Et bien entendu, pour chacune de ces différentes pondérations fréquentielles des termes, il est possible de multiplier ou non par le poids du terme dans le collection $p_{df_{t_i}}$ qui peut être soit $idf_{t_i}$ soit une version normalisée $ max(0, \log{\frac{N -df_{t_i}}{df_{t_i}}})$.

Ces schémas de pondération peuvent être appliqués au document et à la requête. Votre premier travail consiste donc à écrire les fonctions permettant de calculer à partir de l'index inversé et d'informations statistiques additionnelles sur la collection ces différents schémas de pondération.

9- Ecrire une fonction `get_tf (term,doc_ID, index_frequence)` qui permet de récupérer la fréquence d'un terme dans un document à partir d'un index de fréquence.

In [ ]:
from Utils.saveandload_pickle import *
#from Archive_Lab2.Utils.saveandload_pickle import *(colab)

# Chargement de l'index de frequence 
frequence_index = load_inverted_index_pickle('./Index/index_frequence.pickle')
#frequence_index = load_inverted_index_pickle('Archive_Lab2/Index/index_frequence.pickle') (pour colab)

def get_tf(term,doc_ID,index_frequence):
    # A completer
    


In [ ]:
#Test

get_tf("NASSAU", '017',frequence_index) 

10- Ecrire une fonction `get_tf_logarithmique (term,doc_ID, index_frequence)` qui permet de calculer la pondération logarithmique d'un terme. 

In [ ]:
def get_tf_logarithmique (term,doc_ID, index_frequence):
    # A completer
    return

Pour les autres schémas de pondération, il est nécessaire d'avoir au préalable calculé et stocké plusieurs informations statistiques sur la collection comme le nombre total de documents et pour chaque document, la fréquence maximale dans le document, le nombre de termes uniques dans le document ou encore la moyenne des fréquences dans un document. 

11- Ecrire une fonction `get_stats_document(document)` qui pour un document donné sous la forme d'une liste de tokens renvoie un dictionnaire avec les informations statistiques mentionnées ci-dessus (clés : "freq_max", "unique_terms", "freq_moy"). On pourra utiliser la collection `Counter` du module `collections` de la bibliothèque standard de python comme dans le LAB1.

In [ ]:
from collections import Counter

def get_stats_document(document):
    stats={}
    # A completer
    return stats

    

In [ ]:
#Test

text = 'UNITED NATIONS POTENT PYGMY IN MANHATTAN LAST WEEK, THE U.N .GREW TO A TOTAL OF 113 MEMBERS WITH THE ADMISSION OF THE NEWLYINDEPENDENT STATES OF ZANZIBAR AND KENYA . ZANZIBAR CONSISTS OF TWOSMALL ISLANDS IN THE INDIAN OCEAN, WITH A TOTAL POPULATION OF 310,000,OR ABOUT THAT OF OMAHA . NEVERTHELESS, ZANZIBAR HAS ONE VOTE IN THEGENERAL ASSEMBLY, AND IS THUS EQUAL IN VOTING POWER WITH SUCH NUCLEARGIANTS AS THE SOVIET UNION AND THE U.S .'

#A compléter

12- Ecrire une fonction `get_stats_collection(collection)` qui permet de calculer des informations statistiques sur la collection comme le nombre de documents et pour chaque document de la collection, les informations statistiques précédentes.
La fonction retournera un dictionnaire avec les clés "nb_docs" et les doc_ID des documents de la collection. On supposera que la collection passée en paramètre est une collection segmentée et pré-traitée (filtrage et normalisation).



In [ ]:
def get_stats_collection(collection):
    stats={}
    # A completer
    return stats

13- Appliquer cette fonction à la collection TIME afin de construire `stats_collection`.

In [ ]:
from Utils.Lab1 import *

collection_TIME = loadData("./Data/Time/TIME.ALL")

pre_processed_collection_TIME = collection_lemmatize(remove_stop_words(tokenize_Regexp_corpus(collection_TIME),"./Data/Time/TIME.STP"))

stats_collection=# A completer


14- Ecrire une fonction `get_tf_normalise(term,doc_ID, index_frequence,stats_collection)` qui calcule la pondération fréquentielle du terme `term` pour le document d'identifiant `doc_ID` à partir de l'index de fréquence `index_frequence` et d'informations statistiques sur la collection stockée dans le dictionnaire `stats_collection` construit à l'étape précédente.

In [ ]:
def get_tf_normalise(term,doc_ID, index_frequence,stats_collection):
        # A completer
        return 


In [ ]:
#Test

print(get_tf_normalise("NASSAU", '017',frequence_index,stats))

15- Ecrire une fonction `get_tf_logarithme_normalise(term,doc_ID, index_frequence,stats_collection)` qui calcule la pondération fréquentielle logarithmique normalisée du terme `term` pour le document d'identifiant `doc_ID` à partir de l'index de fréquence `index_frequence` et d'informations statistiques sur la collection stockées dans le dictionnaire `stats_collection` construit à l'étape précédente.

In [ ]:
def get_tf_logarithme_normalise(term,doc_ID, index_frequence,stats_collection):
        # A completer
        return 

In [ ]:
# Test

print(get_tf_logarithme_normalise("NASSAU", '017',frequence_index,stats))

16- Ecrire une fonction `get_idf (term ,index_frequence, nb_doc)` qui permet de calculer le logarithme de l'inverse normalisée de la fréquence documentaire $df_t$ d'un terme. `nb_doc`est le nombre de documents dans la collection. Il est égal à 523 pour la collection TIME ou vous pouvez le récupérer via le dictionnaire de statistiques construit.

In [ ]:
from math import *

def get_idf(term,index_frequence,nb_doc):
    # A completer
    return 
               

In [ ]:
#Test

get_idf("NASSAU",frequence_index,523)

**Traitement des requêtes**

17- Ecrire une fonction `processing_vectorial_query(query, inverted_index,stats_collection, weighting_scheme_document,weighting_scheme_query)` qui permet de renvoyer la liste de documents pertinents, ordonnés par pertinence, pour la requête `query` à partir de l'index de fréquence `inverted_index`. Les paramètres `weighting_scheme_document` et `weighting_scheme_query` permettent de définir le type de pondération choisie pour le document et pour la requête. On considérera les types suivants :
 + 'binary' : schema de pondération binaire.
 + 'frequency' : schema de pondération fréquentiel simple (`tf` seul).
 + 'tf_idf_normalize' : schema de pondération `tf_idf` avec la pondération fréquentielle normalisée pour le terme correspondant à la fréquence du terme dans le document.
 + 'tf_idf_logarithmic' : schema de pondération `tf_idf` avec la pondération fréquentielle logarithmique pour le terme correspondant à la fréquence du terme dans le document.
 + 'tf_idf_logarithmic_normalize' : schema de pondération `tf_idf` avec la pondération fréquentielle logarithmique normalisée pour le terme correspondant à la fréquence du terme dans le document.


Vous pourrez pour cela vous appuyer sur l'algorithme décrit dans le support du cours 1 (version longue), slide 136. On ne considérera pour la requête que les schémas de pondération ('binary','frequency') et on prendra comme mesure de similarité le produit scalaire. On ne considérera donc pas de facteur de normalisation.



In [ ]:
def processing_vectorial_query(query, inverted_index, stats_collection, weighting_scheme_document,weighting_scheme_query):
    ordered_relevant_docs = {}
    # a completer
    return ordered_relevant_docs

18- Tester votre modèle.

In [ ]:
# A compléter

## Partie 2 

Vous trouverez sur EDUNAO une série d'exercices à faire en dehors du Lab pour vous entrainer pour l'examen. Une correction sera donnée ultérieurement. 